In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDmodule2 import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 31367
db_name = "AAC"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, db_name)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '100px'}),
        html.H4('Justin Leger CS-340 Dashboard')
    ]),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        
    html.Div([
        html.Label("Filter by Animal Sex:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Male', 'value': 'Male'},
                {'label': 'Female', 'value': 'Female'},
                {'label': 'All', 'value': 'All'}
            ],
            value='All',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    
    html.Hr(),
   
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here
                         
                         page_size=10,
                         page_current=0,
                         page_action="native",
                         sort_action="native",
                         filter_action="native",
                         selected_rows=[0],
                         row_selectable="single",
                         editable=True,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    #
    #        
    #        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    #        data=df.to_dict('records')
    #       
    #       
    #        return (data,columns)
    #set up an if/else if/else block to respond to the radio buttons
    if filter_type == 'All':
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({})) 
    elif filter_type == 'Water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'Mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'Disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }
    else:
        raise Exception("Unknown filter")
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')        
        
    return (data,columns)


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id',"children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    df = pd.DataFrame,from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # If selected_columns is None, use an empty list.
    if selected_columns is None:
        selected_columns = []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
#update geolocation

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Convert the data table's data to a DataFrame.
    dff = pd.DataFrame.from_dict(viewData)
    # Determine which row is selected; default to the first row.
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
   
    #  extract latitude and longitude.
    try:
        lat = float(dff.loc[row, 'location_lat'])
        lon = float(dff.loc[row, 'location_long'])
    except Exception as e:
        # If there is an error, center on default coordinates for Austin, TX.
        lat, lon = 30.75, -97.48
        
    # Austin TX is at [30.75,-97.48]
    # Return a Leaflet map with a marker.
    return dl.Map(style={'width': '1000px', 'height': '500px'},
                  center=[lat, lon], zoom=12, children=[
        dl.TileLayer(),  # Base layer
        dl.Marker(position=[lat, lon],
                  children=[
                      dl.Tooltip(dff.loc[row, 'breed'] if 'breed' in dff.columns else "Unknown Breed"),
                      dl.Popup([
                          html.H1("Animal Name"),
                          html.P(dff.loc[row, 'name'] if 'name' in dff.columns else "Unnamed")
                      ])
                  ])
    ])
   

app.run_server(debug=True)


ServerSelectionTimeoutError: Got response id 3552046 but expected 1433925857, Timeout: 30s, Topology Description: <TopologyDescription id: 68031fae309e9811f1f633cb, topology_type: Single, servers: [<ServerDescription ('nv-desktop-services.apporto.com', 31367) server_type: Unknown, rtt: None, error=ProtocolError('Got response id 3552046 but expected 1433925857')>]>